In [74]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

import random

start_x, end_x = 0, 50
y_position = 25
horizontal_positions = [(x, y_position) for x in range(start_x, end_x)]
vertical_positions = [(25, x) for x in range(start_x, end_x)]
positions = horizontal_positions + vertical_positions


random_goal_position = random.choice(positions)
print("random_goal_position : ", random_goal_position)

class MovingAgent(ap.Agent):
    def setup(self):
        self.position = (25, 0)  # Starting position
        self.goal = random_goal_position    # Goal position
        self.path = positions
        # self.condition = 1

    def move(self):
        # Define movement logic here to move towards the goal
        # Ensure movement is within allowed positions
        # If goal is reached, stop the model
        if self.position == self.goal:
            self.model.stop()
        else:
            # Example movement: move closer to the goal
            x, y = self.position
            goal_x, goal_y = self.goal

            # Move in the x direction
            if x < goal_x and (x + 1, y) in self.path:
                x += 1
            elif x > goal_x and (x - 1, y) in self.path:
                x -= 1

            # Move in the y direction
            if y < goal_y and (x, y + 1) in self.path:
                y += 1
            elif y > goal_y and (x, y - 1) in self.path:
                y -= 1

            self.position = (x, y)

#! specific position on the grid, you should use self.forest.agents[position]
#* คอมเมนท์สีเขียวหมายความว่าเคยใช้งาน แต่เอาออกไปแล้ว
#TODO คอมเมนท์สีส้มคือ ทำภายในอนาคต
#? คอมเมนท์สีฟ้าเอาไว้บอกส่วนสำคัญเฉยๆ
# คอมเมนท์สีเทา ไว้อธิบายเฉย


class Road(ap.Agent):
    def setup(self):
        # `position` เป็นคุณสมบัติของเอเยนต์
        self.position = None


class ForestModel(ap.Model):

    def setup(self):

        # Create agents (trees)
        #* n_trees = int(self.p['Tree density'] * (self.p.size**2))
        n_trees = 100
        roads = self.roads = ap.AgentList(self, n_trees , Road) # สร้างตัวแทน n_trees ตัว



        # Create grid (forest) สร้างกริดขนาด 50*50
        self.map = ap.Grid(self, [self.p.size]*2, track_empty=True) # https://agentpy.readthedocs.io/en/latest/reference_grid.html#agentpy.Grid
        # ต้นไม้มี 1500 พื้นที่ทั้งหมดบนแมพมี 2500 (50*50)


        # TODO ทำ Position
        # position เป็น array --> [(0, 25), (1, 25) , ...] 
        # เอาไว้ให้รู้ว่า ถ้าอยากฟิกตำแหน่ง agent ตัวที่ 50 มันทำไง
        # positions = [(i % 50, i // 50) for i in range(2500)]
        # positions[50] = (2, 8)


        self.map.add_agents(roads, positions=positions, empty=True) # https://agentpy.readthedocs.io/en/latest/reference_environments.html

        #? Initialize conditions 
        self.roads.condition = 0
        # condition เป็นค่าที่คุณกำหนดให้เฉพาะกลุ่มของเอเยนต์ 
        # condition ของเอเยนต์ทั้งหมดใน self.roads เป็น 0 แต่ถ้าเอเยนต์ใดไม่ได้อยู่ใน self.roads มันจะมีค่า None




        #? Define positions
        # self.positions = positions
        self.map.add_agents(roads, positions=positions, empty=True) # add Road Agent
        for road, pos in zip(self.roads, positions):
            road.position = pos
            # -->  Road (Obj 1) pos : (0, 25)
        # for agent in self.map.agents:
        #     print(f"agent {agent.position} condition : {agent.condition}")
        # agent (0, 25) condition : 0


        #? Define function to get agents at specific positions
        # def get_agents_at_position(grid, position):
        #     return [agent for agent in grid.agents if grid.positions[agent] == position]

        
        #? Create a MovingAgent 
        bicycle = self.bicycle = ap.AgentList(self, 1, MovingAgent)
        self.map.add_agents(bicycle, positions=bicycle.position)
        self.bicycle.condition = 3

        
        #? Set Goal
        reachTarget = self.map.agents[random_goal_position]  # Access agents at position (10, 25)
        # reachTarget เป็น GridIter 
        print(f"reachTarget: {reachTarget}")
        if reachTarget:  # Check if any agents exist at that position
            for agent in reachTarget:  # Loop through agents and update condition
                print(f"agent : {agent}")
                if isinstance(agent, Road):
                    agent.condition = 4
        else:
            print("No agent found at (10, 25)")

        # reachTarget: GridIter (2 objects)
        # agent : Road (Obj 51)
        # agent : MovingAgent (Obj 102)


        # print("self.map.agents : ",  self.map.agents) # GridIter (101 objects)

        

        print("----------------------- end setup")

    def step(self):

        # Select burning trees
        #* burning_trees = self.agents.select(self.agents.condition == 1) # มี AgentList (0 objects) ที่ได้งี้ เพราะเราไม่ใส่ condition 1 ลงไป
        # for tree in burning_trees:
        #     for neighbor in self.forest.neighbors(tree):



        # class AgentList ทั้งสองตัว
        find_bike = self.bicycle.select(self.bicycle.condition == 3) # มี AgentList (1 object)
        start = self.roads.select(self.roads.position == find_bike.position[0])  # Find the starting position
        
        for gridRoad in start:
            # gridRoad class Road
            for neighbor in self.map.neighbors(gridRoad):
                if neighbor.position == gridRoad.position:
                    continue
                else:
                    # self.map.move_by(find_bike, neighbor.position)
                    find_bike.position = neighbor.position
                    break; # breakเพราะเพื่อนบ้านมีหลายอัน แต่เราต้องการอันเดียวแล้วจบเลย


        # Concept
        # for ตำแหน่งจุดหนึ่งที่มีจักรยานอยู่gridRoad(Agent) in ถนนทั้งหมดRoads(AgentList)
        #     for เพื่อนบ้านของตำแหน่งกริดถนนที่มีจักรยานอยู่ in เพื่อนบ้านของgridRoad(AgentIter)
        #         # มันจะมีเพื่อนบ้าน 4 ทิศ เขียนเงื่อนไข
        #         ถ้าเป็นตำแหน่งตัวเอง:
        #             continue
        #         moving
            
            


        
        # print("start : ",start) # ⁡⁢⁢⁣ensure that start refers to a single agent, not an AgentList or collection of agents⁡
        # start :  AgentList (1 object)
        # print(" start[0]", start.position) # --> [(25, 0)]
        # print(" start[0]", start) # --> Road (Obj 51) -- เอเยนต์ Road นี้เป็นเอเยนต์ที่ 51 ที่ถูกสร้างขึ้น
        # print(f"type : {type(start)}") # --> AgentList
        # for road in self.roads:
        #     print(road.position)

        # burning_trees = self.agents.select(self.agents.condition == 1)
        # road_path = self.roads.select(self.roads.condition == 0)
        # for road in self.roads:
        #     print(type(road))




        # # !!!! จะกลับมาทำ step ต่อ 👀👀👀 !!!!
        # for agent_bike in find_bike:
        #     neighbors = self.map.neighbors(start) # ต้องเป็น AgentIter
        #     for neighbor in neighbors:
        #         print(neighbor)



            #     print("neighbor.condition == 0 " , neighbor)
            #     if neighbor.position == agent_bike.position:
            #         print("continue 🌶️")
            #         continue

                
            #     # Move agent to neighbor if condition == 4
            #     if neighbor.condition == 4:
            #         self.map.move_by(agent_bike, neighbor.position)
            #         self.stop()
            #         break
            #     # elif neighbor.condition == 0 and neighbor.position != agent_bike.position:
            #     # #     #TODO ยังไม่สนเรื่องถอยหลังหรือหน้า แค่ขยับออกจากตรงที่มันอยู่ก่อน --> ไปทำต่อในอนาคต
            #     #     print("ก่อน move", agent_bike.position[0])
            #     #     self.map.move_by(agent_bike, neighbor.position)
            #     #     print("หลัง move", agent_bike.position[0])
            #     #     # แก้ไข position ของ bike ให้เป็นตำแหน่งใหม่
            #     # #     neighbor.condition = 3
            #     elif neighbor.condition == 0 and neighbor.position != agent_bike.position:
            #         # ขยับจักรยาน
            #         print("𓆝 𓆟 𓆞 𓆝 𓆟")
            #         print("Before move", agent_bike.position)
            #         self.map.move_by(agent_bike, neighbor.position)
            #         # print(f"Neighbors of agent at position {neighbor.position} condition is : {neighbor.condition}") # มันได้ 2 เพราะได้ตำแหน่ง (25, 0) กับ (25, 1)  คือมันนับตำแหน่งปัจจุบันมันด้วยอะ
                    
            #         # เปลี่ยนตำแหน่งจักรยาน
            #         agent_bike.position = neighbor.position 
            #         print("After move", agent_bike.position)
            #         print("𓆝 𓆟 𓆞 𓆝 𓆟")

            #         # เปลี่ยน condition 3 ให้เท่ากับตำแหน่งใหม่


        if self.t == 10:
            self.stop()


        
        # move_next = self.forest.neighbors(self.bicycle)
        # print(mo)
        # grid.move_agent(agents[agent_index], target_position)


        # Spread fire
        # for tree in burning_trees:
        #     for neighbor in self.forest.neighbors(tree):
        #         # attribute position จะมีแค่ที่เรากำหนดลงให้ คือ goal กับ จุดเริ่มต้น
        #         if neighbor.condition == 0:
        #             neighbor.condition = 1 # Neighbor starts burning
        #     tree.condition = 2 # Tree burns out
        #     # print(f"------------------ Step: {self.t} ------ {(len(self.forest.neighbors(tree)))}")

        # # Stop simulation if no fire is left
        # if len(burning_trees) == 0:
        #     self.stop()
        
        

        
        




    def end(self):

        # Document a measure at the end of the simulation
        # burned_trees = len(self.agents.select(self.agents.condition == 2))
        # self.report('Percentage of burned trees',
        #             burned_trees / len(self.agents))
        self.report("Bicycle Sharing")

        
        
parameters = {
    'Tree density': 0.2, # Percentage of grid covered by trees
    'size': 50, # Height and length of the grid
    'steps': 100,
}

# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.map.attr_grid('condition') # attr_grid() เป็นฟังก์ชันที่ใช้เรียกค่าแอตทริบิวต์ (attribute) ของตัวแทนทั้งหมดในพื้นที่ grid ของป่า
    # ฟังก์ชันด้านบน attr_grid ด้านบน จะสร้าง grid (ตารางหรือแผนที่) ของป่าที่แสดงสถานะของตัวแทน (agents) ทุกตัว โดยดึงข้อมูลจากแอตทริบิวต์ condition ของแต่ละตัวแทนและจัดเรียงเป็นรูปแบบ grid
    # A two-dimensional grid (กริดสองมิติ) คือโครงสร้างข้อมูลที่ใช้ในการจัดระเบียบข้อมูลหรือวัตถุต่างๆ ลงในรูปแบบของตารางที่มีสองมิติ—มิติแถวและมิติคอลัมน์ โดยทั่วไปแล้วจะมีลักษณะเป็นตารางที่แบ่งออกเป็นเซลล์ (cells) ที่แต่ละเซลล์สามารถเก็บข้อมูลหรือวัตถุได้
    
    # 0 คือ เริ่มต้น น้ำเงินนนน คือถนน ,1 คือถนน ดำำำ,  2 คือเป็นที่ที่เดินแล้ว ชมพู , None คือสีน้ำตาล ,3 คือ actor or bicycle , 4 คือ goal , '#d5e5d5' หรือ None อันก่อนเป็นพื้นที่ที่ไม่ใช่ถนน
    color_dict = {0:'#0000FF',  2:'#ff00fb', None:'#402b2f' , 3:"#aa7eee", 4:'#ffeb00'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True) # https://agentpy.readthedocs.io/en/latest/reference_visualization.html
    ax.set_title(f"Simulation\n"
                 f"Time-step: {model.t}"
                #  f"Trees left: {len(model.agents.select(model.agents.condition == 0))}"
                )

fig, ax = plt.subplots()
model = ForestModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))


random_goal_position :  (42, 25)
reachTarget: GridIter (1 object)
agent : Road (Obj 43)
----------------------- end setup
gridRoad <class '__main__.Road'>
gridRoad <class '__main__.Road'>
gridRoad <class '__main__.Road'>
gridRoad <class '__main__.Road'>
gridRoad <class '__main__.Road'>
gridRoad <class '__main__.Road'>
gridRoad <class '__main__.Road'>
gridRoad <class '__main__.Road'>
gridRoad <class '__main__.Road'>
gridRoad <class '__main__.Road'>
